# 1D-CNN Model for ECG Classification
- The model used has 2 Conv. layers and 2 FC layers.
- This code repeat running the training process and produce all kinds of data which can be given, such as data needed for drawing loss and accuracy graph through epochs, and maximum test accuracy for each run.

## Get permission of Google Drive access

In [1]:
root_path = '../../'

## File name settings

In [2]:
data_dir = 'mitdb'
train_name = 'train_ecg.hdf5'
test_name = 'test_ecg.hdf5'
all_name = 'all_ecg.hdf5'

model_dir = 'model'
model_name = 'conv2'
model_ext = '.pth'

csv_dir = 'csv'
csv_ext = '.csv'

csv_name = 'conv2'
csv_accs_name = 'accs_conv2'

## Import required packages

In [3]:
mode = 'CIFAR10'

import os

import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
from torch.optim import Adam

import numpy as np
import pandas as pd
import h5py
import matplotlib.pyplot as plt
import torchvision
from torchvision.datasets import CIFAR10
from torch.utils.data import DataLoader

## GPU settings

In [4]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
if torch.cuda.is_available():
    print(torch.cuda.get_device_name(0))

NVIDIA GeForce RTX 4070


## Define `ECG` `Dataset` class


In [5]:

class ECG(Dataset):
    def __init__(self, mode='train'):
        if mode == 'train':
            with h5py.File(os.path.join(root_path, data_dir, train_name), 'r') as hdf:
                self.x = hdf['x_train'][:]
                self.y = hdf['y_train'][:]
        elif mode == 'test':
            with h5py.File(os.path.join(root_path, data_dir, test_name), 'r') as hdf:
                self.x = hdf['x_test'][:]
                self.y = hdf['y_test'][:]
        elif mode == 'all':
            with h5py.File(os.path.join(root_path, data_dir, all_name), 'r') as hdf:
                self.x = hdf['x'][:]
                self.y = hdf['y'][:]
        else:
            raise ValueError('Argument of mode should be train, test, or all.')
    
    def __len__(self):
        return len(self.x)
    
    def __getitem__(self, idx):
        return torch.tensor(self.x[idx], dtype=torch.float), torch.tensor(self.y[idx])

if mode == 'ECG':
    batch_size = 32
    train_dataset = ECG(mode='train')
    test_dataset = ECG(mode='test')
    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)
    x_train, y_train = next(iter(train_loader))
    print(x_train.size())
    print(y_train.size())
    total_batch = len(train_loader)
    print(total_batch)
elif mode == 'CIFAR10':
    batch_size = 32

    root_path = '../../data/'
    batch_size = 32

    cifar10_mean = (0.485, 0.456, 0.406)
    cifar10_std = (0.229, 0.224, 0.225)

    transform = torchvision.transforms.Compose([
        torchvision.transforms.ToTensor(),
        torchvision.transforms.Normalize(mean=cifar10_mean, std=cifar10_std)
    ])
    train_dataset = CIFAR10(root=root_path, train=True, download=True, transform=transform)
    test_dataset = CIFAR10(root=root_path, train=False, download=True, transform=transform)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=False)
    test_loader = DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    print("Loaded CIFAR10 datasets with batch size", batch_size)

Files already downloaded and verified
Files already downloaded and verified
Loaded CIFAR10 datasets with batch size 32


In [6]:
from torchvision.models import resnet18, vgg16

if mode == 'ECG':
    class ECGConv(nn.Module):
        def __init__(self):
            super(ECGConv, self).__init__()
            self.conv1 = nn.Conv1d(1, 16, 7, padding=3)  # 128 x 16
            self.relu1 = nn.ReLU()
            self.pool1 = nn.MaxPool1d(2)  # 64 x 16
            self.conv2 = nn.Conv1d(16, 16, 5, padding=2)  # 64 x 16
            self.relu2 = nn.ReLU()
            self.pool2 = nn.MaxPool1d(2)  # 32 x 16
            self.linear3 = nn.Linear(32 * 16, 128)
            self.relu3 = nn.ReLU()
            self.linear4 = nn.Linear(128, 5)
            self.softmax4 = nn.Softmax(dim=1)
        
        def forward(self, x):
            x = self.conv1(x)
            x = self.relu1(x)
            x = self.pool1(x)
            x = self.conv2(x)
            x = self.relu2(x)
            x = self.pool2(x)
            x = x.view(-1, 32 * 16)
            x = self.linear3(x)
            x = self.relu3(x)
            x = self.linear4(x)
            x = self.softmax4(x)
            return x

elif mode == 'CIFAR10':
    NUM_CLASSES = 10

    class CIFAR10VGG(nn.Module):
        def __init__(self):
            super(CIFAR10VGG, self).__init__()
            # Load pre-trained VGG16 model
            self.model = vgg16(pretrained=True)

            # Replace the classifier's last layer to match CIFAR10 classes (10 classes)
            num_features = self.model.classifier[6].in_features
            self.model.classifier[6] = nn.Linear(num_features, 10)

        def forward(self, x):
            return self.model(x)
    
    class AlexNet(nn.Module):
        def __init__(self, num_classes=NUM_CLASSES):
            super(AlexNet, self).__init__()
            self.features = nn.Sequential(
                nn.Conv2d(3, 64, kernel_size=3, stride=2, padding=1),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(kernel_size=2),
                nn.Conv2d(64, 192, kernel_size=3, padding=1),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(kernel_size=2),
                nn.Conv2d(192, 384, kernel_size=3, padding=1),
                nn.ReLU(inplace=True),
                nn.Conv2d(384, 256, kernel_size=3, padding=1),
                nn.ReLU(inplace=True),
                nn.Conv2d(256, 256, kernel_size=3, padding=1),
                nn.ReLU(inplace=True),
                nn.MaxPool2d(kernel_size=2),
            )
            self.classifier = nn.Sequential(
                nn.Dropout(),
                nn.Linear(256 * 2 * 2, 4096),
                nn.ReLU(inplace=True),
                nn.Dropout(),
                nn.Linear(4096, 4096),
                nn.ReLU(inplace=True),
                nn.Linear(4096, num_classes),
            )

        def forward(self, x):
            x = self.features(x)
            x = x.view(x.size(0), 256 * 2 * 2)
            x = self.classifier(x)
            return x
        
    # class ECGConv(nn.Module):
    #     def __init__(self):
    #         super(ECGConv, self).__init__()
    #         self.conv1 = nn.Conv2d(3, 16, 7, padding=3)
    #         self.relu1 = nn.LeakyReLU()
    #         self.pool1 = nn.MaxPool2d(2)
    #         self.conv2 = nn.Conv2d(16, 16, 5, padding=2)
    #         self.relu2 = nn.LeakyReLU()
    #         self.pool2 = nn.MaxPool2d(2)
    #         self.linear3 = nn.Linear(8 * 8 * 16, 128)
    #         self.relu3 = nn.LeakyReLU()
    #         self.linear4 = nn.Linear(128, 10)
    #         self.softmax4 = nn.Softmax(dim=1)
        
    #     def forward(self, x):
    #         x = self.conv1(x)
    #         x = self.relu1(x)
    #         x = self.pool1(x)
    #         x = self.conv2(x)
    #         x = self.relu2(x)
    #         x = self.pool2(x)
    #         x = x.view(-1, 8 * 8 * 16)
    #         x = self.linear3(x)
    #         x = self.relu3(x)
    #         x = self.linear4(x)
    #         x = self.softmax4(x)
    #         return x

    # class ECGConv(nn.Module):
    #     def __init__(self):
    #         super(ECGConv, self).__init__()
    #         # Load pre-trained ResNet18 model
    #         self.model = resnet18()

    #         num_ftrs = self.model.fc.in_features
    #         self.model.fc = nn.Linear(num_ftrs, 10)  # Replace num_output_classes with your number of classes

    #     def forward(self, x):
    #         return self.model(x)

## Training process settings

In [7]:
run = 1
epoch = 400
lr = 0.001

## Traning function


In [8]:
def train(nrun, model):
    criterion = nn.CrossEntropyLoss()
    optimizer = Adam(model.parameters(), lr=lr)

    train_losses = list()
    train_accs = list()

    test_losses = list()
    test_accs = list()

    for e in range(epoch):
        print("Epoch {} - ".format(e+1), end='')

        # train
        train_loss = 0.0
        correct, total = 0, 0
        for _, batch in enumerate(train_loader):
            x, label = batch  # get feature and label from a batch
            x, label = x.to(device), label.to(device)  # send to device
            optimizer.zero_grad()  # init all grads to zero
            output = model(x)  # forward propagation
            loss = criterion(output, label)  # calculate loss
            loss.backward()  # backward propagation
            optimizer.step()  # weight update

            train_loss += loss.item()
            correct += torch.sum(output.argmax(dim=1) == label).item()
            total += len(label)
        train_losses.append(train_loss / len(train_loader))
        train_accs.append(correct / total)
        print("loss: {:.4f}, acc: {:.2f}%".format(train_losses[-1], train_accs[-1]*100), end=' / ')
        
        # test
        with torch.no_grad():
            test_loss = 0.0
            correct, total = 0, 0
            for _, batch in enumerate(test_loader):
                x, label = batch
                x, label = x.to(device), label.to(device)
                output = model(x)
                loss = criterion(output, label)
                
                test_loss += loss.item()
                correct += torch.sum(output.argmax(dim=1) == label).item()
                total += len(label)
            test_losses.append(test_loss / len(test_loader))
            test_accs.append(correct / total)
        print("test_loss: {:.4f}, test_acc: {:.2f}%".format(test_losses[-1], test_accs[-1]*100))

    return train_losses, train_accs, test_losses, test_accs

## Training process

### Repeat for 10 times

In [9]:
best_test_accs = list()

for i in range(run):
    print('Run', i+1)
    seed = 0
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.benchmark = False
    torch.backends.cudnn.deterministic = True
    
    ecgnet = AlexNet()
    torch.save(ecgnet.state_dict(), f'init_weight{mode}.pth')
    train_losses, train_accs, test_losses, test_accs = train(i, ecgnet.to(device))

    best_test_accs.append(max(test_accs))
    best_test_acc_epoch = np.array(test_accs).argmax() + 1
    print('Best test accuracy {:.2f}% in epoch {}.'.format(best_test_accs[-1]*100, best_test_acc_epoch))
    print('-' * 100)

    df = pd.DataFrame({
        'loss': train_losses,
        'test_loss': test_losses,
        'acc': train_accs,
        'test_acc': test_accs
    })
    df.to_csv(os.path.join(root_path, csv_dir, '_'.join([csv_name, str(i+1)]) + csv_ext))

df = pd.DataFrame({'best_test_acc': best_test_accs})
df.to_csv(os.path.join(root_path, csv_dir, csv_accs_name + csv_ext))

Run 1
Epoch 1 - loss: 1.6942, acc: 35.12% / test_loss: 1.4396, test_acc: 46.39%
Epoch 2 - loss: 1.3385, acc: 51.79% / test_loss: 1.2352, test_acc: 56.07%
Epoch 3 - loss: 1.1839, acc: 58.13% / test_loss: 1.1567, test_acc: 60.08%
Epoch 4 - loss: 1.0857, acc: 62.14% / test_loss: 1.1251, test_acc: 61.69%
Epoch 5 - loss: 1.0030, acc: 65.16% / test_loss: 1.1095, test_acc: 62.57%
Epoch 6 - loss: 0.9434, acc: 67.49% / test_loss: 1.1073, test_acc: 63.55%
Epoch 7 - loss: 0.8945, acc: 69.27% / test_loss: 1.0815, test_acc: 65.63%
Epoch 8 - loss: 0.8495, acc: 70.68% / test_loss: 1.1307, test_acc: 64.33%
Epoch 9 - loss: 0.8036, acc: 72.21% / test_loss: 1.1326, test_acc: 63.43%
Epoch 10 - loss: 0.7810, acc: 73.25% / test_loss: 1.1247, test_acc: 65.12%
Epoch 11 - loss: 0.7624, acc: 73.78% / test_loss: 1.1534, test_acc: 66.07%
Epoch 12 - 

KeyboardInterrupt: 

## Print the best test accuracy of each run

In [ ]:
for i, a in enumerate(best_test_accs):
    print('Run {}: {:.2f}%'.format(i+1, a*100))

In [ ]:
df = pd.read_csv(os.path.join('csv', 'conv2_1.csv'))
test_accs = df['test_acc']
train_accs = df['acc']
test_losses = df['test_loss']
train_losses = df['loss']

fig, ax = plt.subplots(1, 2, figsize=(15, 4))

ax[0].plot(train_losses)
ax[0].plot(test_losses)
ax[0].set_xticks([0, 100, 200, 300, 400])
ax[0].set_xlabel('Epoch', size=16)
ax[0].set_ylabel('Loss', size=16)
ax[0].set_ylim(0.9, 1.1)
ax[0].set_yticks([0.9, 1.0, 1.1, 1.2])
ax[0].grid(alpha=0.5)
ax[0].tick_params(labelsize=16)
ax[0].legend(['Train', 'Test'], loc='right', fontsize=16)

ax[1].set_ylim(0.7, 1.0)
ax[1].set_yticks([0.7, 0.8, 0.9, 1.0])
ax[1].plot(train_accs)
ax[1].plot(test_accs)
yt = ax[1].get_yticks()
ax[1].set_yticklabels(['{:,.0%}'.format(x) for x in yt])
ax[1].set_xticks([0, 100, 200, 300, 400])
ax[1].set_xlabel('Epoch', size=16)
ax[1].set_ylabel('Accuracy', size=16, labelpad=-5)
ax[1].grid(alpha=0.5)
ax[1].tick_params(labelsize=16)
ax[1].legend(['Train', 'Test'], loc='right', fontsize=16)

fig.savefig('loss_acc_conv2_seed.pdf', bbox_inches='tight')

FileNotFoundError: [Errno 2] No such file or directory: 'csv\\conv2_1.csv'